<a href="https://colab.research.google.com/github/ncmiquel/03MIAR---Algoritmos-de-Optimizacion/blob/main/Algoritmos_AG2_Ignacio_Carrillo_Miquel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AG2 - Actividad Guiada 2
Nombre: Ignacio Carrillo Miquel <br>
Link:   https://colab.research.google.com/drive/1JENXagpq-Lyla01DHuqHnwdwo4xvtkyO?usp=sharing <br>
Github: https://github.com/ncmiquel/03MIAR---Algoritmos-de-Optimizacion
<br>




In [1]:
import math

## PROGRAMACIÓN DINÁMICA

Vamos a tratar de resolver el probelma del río: <br>
En un río hay n embarcaderos y debemos desplazarnos río abajo desde un embarcadero a otro. Cada embarcadero tiene precios diferentes para ir de uno a otro. Para ir del embarcadero i al j, puede ocurrir que sea más barato hacer un trasbordo por un embarcadero intermedio k. El problema consiste en determinar la combinación más barata. <br>

En primer lugar, trasladaremos las tarifas de cada embarcadero a una matriz de tarifas. Los valores marcados como 999, están para garantizar que ese trayecto no se va a elegir para incluirlo en la ruta óptima.



In [2]:
# Viaje por el rio - Programación dinámica

# Constante para indicar que no hay tarifa disponible
NT = math.inf

# Tarifas entre los embarcaderos
tarifas = [
[0, 5, 4, 3, NT, NT, NT],      # desde nodo 0
[NT, 0, NT, 2, 3, NT, 11],     # desde nodo 1
[NT, NT, 0, 1, NT, 4, 10],     # desde nodo 2
[NT, NT, NT, 0, 5, 6, 9],      # desde nodo 3
[NT, NT, NT, NT, 0, NT, 4],    # desde nodo 4
[NT, NT, NT, NT, NT, 0, 3],    # desde nodo 5
[NT, NT, NT, NT, NT, NT, 0]    # desde nodo 6
]


In [3]:
# Calcularemos la matriz de PRECIOS y RUTAS

# precios: contiene la matriz del mejor precio para ir de un nodo a otro
# ruta: contiene los nodos intermedios para ir de un nodo a otro

def Precios(tarifas):
  # Total de Nodos
  N = len(tarifas[0])
  
  # Inicializamos la tabla de precios
  precios = [[NT]*N for i in range(N)]  # n x n 
  ruta = [[-1]*N for i in range(N)]
  
  # Recorremos todos los nodos con dos bucles(origen - destino) para ir construyendo la matriz de PRECIOS
  
  # Bucle para embarcaciones de origen
  for i in range(N-1):
    # Bucle para embarcaciones de destino
    for j in range(i+1, N):
      # Inicializamos el precio MIN y la ruta con los valores directos
      precio_minimo = tarifas[i][j]
      ruta[i][j] = i
      # Bucle para embarcaciones intermedias
      for k in range(i, j):
        # Si hay una ruta más económica a través de un embarcadero intermedio, actualizamos el precio y la ruta
        precio_intermedio = precios[i][k] + tarifas[k][j]
        if precio_intermedio < precio_minimo:
            precio_minimo = precio_intermedio
            ruta[i][j] = k
        precios[i][j] = precio_minimo
        
  return precios,ruta

In [4]:
precios,ruta = Precios(tarifas)  

print("PRECIOS")
display(precios)

print("\nRUTA")  
display(ruta) 

PRECIOS


[[inf, 5, 4, 3, 8, 8, 11],
 [inf, inf, inf, 2, 3, 8, 7],
 [inf, inf, inf, 1, 6, 4, 7],
 [inf, inf, inf, inf, 5, 6, 9],
 [inf, inf, inf, inf, inf, inf, 4],
 [inf, inf, inf, inf, inf, inf, 3],
 [inf, inf, inf, inf, inf, inf, inf]]


RUTA


[[-1, 0, 0, 0, 1, 2, 5],
 [-1, -1, 1, 1, 1, 3, 4],
 [-1, -1, -1, 2, 3, 2, 5],
 [-1, -1, -1, -1, 3, 3, 3],
 [-1, -1, -1, -1, -1, 4, 4],
 [-1, -1, -1, -1, -1, -1, 5],
 [-1, -1, -1, -1, -1, -1, -1]]

In [5]:
#Calculo de la ruta usando la matriz RUTA
def calcular_ruta(RUTA, desde, hasta):
  if desde == RUTA[desde][hasta]:
    return desde 
  else:
    return str(calcular_ruta(RUTA, desde, RUTA[desde][hasta])) +  ',' + str(RUTA[desde][hasta]) 

print("\nLa ruta es:")  
calcular_ruta(ruta, 0,6) 


La ruta es:


'0,2,5'

## RAMIFICACIÓN Y PODA

### Problema de asignación de tarea
En este caso, tenemos que minimizar los costes de asignación de N tareas a N agentes. Cada tarea solo puede ser
asignada a un agente.

In [6]:
# Asignacion de tareas (Ramificación y Poda)

# Presentamos la matriz de costes (Brassard p.349)
# Cada agente es una fila. Cada tarea es una columna.

COSTES=[[11,12,18,40],
        [14,15,13,22],
        [11,17,19,23],
        [17,14,20,28]]
        

Definimos la función para calcular el valor de una solución dada:

In [7]:
# Valor de una solucion parcial

# Sirve para ir calculando el valor de las soluciones 
# (las posiciones 0, 1, 2, 3 son los agentes)
# (Los valores de S(solución): (1, 2, 3, 4) son las tareas asignadas a cada agente)
def valor(S,COSTES):
  VALOR = 0
  for i in range(len(S)):
    VALOR += COSTES[S[i]][i] 
  return VALOR

valor((3,2,1,0),COSTES)

87

**IMPORTANTE**: hemos hecho cambios para mejorar la función de cálculo de costes:
1. Hemos unido en una sola función el cálculo de la cota máxima y mínima (según se especifique 'min' o 'max').
2. Hemos añadido un control booleano (tarea not in S) para asegurarnos de que solo añadimos los valores que no hemos sumado previamente.

In [8]:
# Coste para soluciones parciales
#  (1,3,) Se asigna la tarea 1 al agente 0 y la tarea 3 al agente 1

def COSTE(S, COSTES, minmax):
    VALOR = 0
     #Valores establecidos

    for tarea in range(len(S)):
        VALOR += COSTES[tarea][S[tarea]]
    #Estimacion
    for tarea in range(len(COSTES)):
        if minmax == 'max' and tarea not in S:
            VALOR += max([COSTES[agente][tarea] for agente in range(len(S), len(COSTES))])
        if minmax == 'min' and tarea not in S:
            VALOR += min([COSTES[agente][tarea] for agente in range(len(S), len(COSTES))])
    
    return VALOR

COSTE((0,),COSTES,'min')

60

La siguiente función crea los hijos que faltan a partir de una solución:

In [9]:
#Genera tantos hijos como como posibilidades haya para la siguiente elemento de la tupla
#(0,) -> (0,1), (0,2), (0,3)

def crear_hijos(NODO, N):
  HIJOS = []
  for i in range(N):
    if i not in NODO:
      HIJOS.append({'s':NODO + (i,)})
  return HIJOS

crear_hijos((0,2,3),4)

[{'s': (0, 2, 3, 1)}]

Por último presentamos el algoritmo de ramificación y poda, aplicado para este caso de asignación de tareas al menor coste:

In [10]:
def ramificacion_y_poda(COSTES):
  # Construccion iterativa de soluciones(arbol). En cada etapa asignamos un agente(ramas).
  # Nodos del grafo  {s:(1,2),CI:3,CS:5}

  # Inicializamos variables
  DIMENSION = len(COSTES)  
  MEJOR_SOLUCION = tuple(i for i in range(len(COSTES)))
  CotaSup = valor(MEJOR_SOLUCION, COSTES)
  NODOS=[]
  NODOS.append({'s':(), 'ci':COSTE((),COSTES, 'min')})
  iteracion = 0

  while(len(NODOS) > 0):
    iteracion +=1

    nodo_prometedor = [min(NODOS, key=lambda x:x['ci']) ][0]['s']

    # Ramificacion
    # Se generan los hijos
    HIJOS =[{'s':x['s'], 'ci':COSTE(x['s'], COSTES, 'min')} for x in crear_hijos(nodo_prometedor, DIMENSION)]

    # Revisamos la cota inferior y nos quedamos con la mejor solucion si llegamos a una solucion final
    NODO_FINAL = [x for x in HIJOS if len(x['s']) == DIMENSION]
    if len(NODO_FINAL ) >0: 
      if NODO_FINAL[0]['ci'] < CotaSup:
        CotaSup = NODO_FINAL[0]['ci']
        MEJOR_SOLUCION = NODO_FINAL
 
    # Poda
    HIJOS = [x for x in HIJOS if x['ci'] < CotaSup]

    # Añadimos los hijos 
    NODOS.extend(HIJOS) 

    # Eliminamos el nodo ramificado
    NODOS =  [  x for x in NODOS if x['s'] != nodo_prometedor]
   
  print("La solucion final es:", MEJOR_SOLUCION, "en", iteracion, "iteraciones", "para dimension:", DIMENSION)

ramificacion_y_poda(COSTES)

La solucion final es: [{'s': (0, 2, 3, 1), 'ci': 61}] en 14 iteraciones para dimension: 4


Como vemos, la solución coincide con lo esperado para este caso.

## AMPLIACIÓN

- Generar matrices con valores aleatorios de mayores
dimensiones (5,6,7,…) y ejecutar ambos algoritmos.
- ¿A partir de que dimensión el algoritmo por fuerza bruta
deja de ser una opción?
- ¿Hay algún valor de la dimensión a partir de la cual el
algoritmo de ramificación y poda también deja de ser una
opción válida? <br>

En primer lugar, calcularemos la asignación de costes por fuerza bruta, es decir, calcularemos todas las posibles soluciones y evaluaremos el coste de cada una de ellas. Nos quedaremos con la de menor coste, que debe coincidir con la que ya hemos obtenido. Luego aplicaremos los algoritmos a matrices de mayores dimensiones, tal y cómo se propone en el enunciado.

In [11]:
import itertools
import math

def coste_fuerza_bruta(costes):
  # Obtenemos cuántas tareas y agentes tenemos
  dimension = len(costes)  
  # Obtenemos el número de combinaciones que tenemos para la dimensión establecida (con itertools)
  combinaciones = list(itertools.permutations(range(dimension), dimension))
  coste_todas_soluciones = []
  # Obtenemos el coste para cada solución
  for solucion in combinaciones:
    coste_todas_soluciones = coste_todas_soluciones + [{'s' : solucion, 'ci' : COSTE(solucion, costes, 'min')}]

  #Revisamos las cotas inferiores y nos quedamos con la mejor solucion
  solucion_optima = [{'s': [min(coste_todas_soluciones, key=lambda x:x['ci'])][0]['s'], 'ci': [min(coste_todas_soluciones, key=lambda x:x['ci'])][0]['ci']}]
  print("La solucion final es:", solucion_optima, "en", math.factorial(dimension), "iteraciones" , "para dimension:", dimension)

coste_fuerza_bruta(COSTES)

La solucion final es: [{'s': (0, 2, 3, 1), 'ci': 61}] en 24 iteraciones para dimension: 4


Como vemos, por fuerza bruta llegamos a la misma solución. Vamos a probarlo ahora con matrices de mayores dimensiones. Así pues, tenemos:

In [12]:
import random
import pandas as pd
import time

# Probamos los algoritmos por fuerza bruta y por ramificación y poda, y determinamos los tiempos de ejecución

# Creamos un DataFrame vacío para guardar los tiempos de ejecución
df = pd.DataFrame()
# Generamos matrices cuadradas aleatorias de tamaño n x n para 5 ≤ n ≤ 8
for n in range(5, 9):
    # Generamos una matriz aleatoria de tamaño n x n con números del 1 al 50
    matriz = [[random.randint(1, 51) for j in range(n)] for i in range(n)]
    
    # Medimos el tiempo de ejecución del algoritmo de ramificación y poda
    print('Algoritmo de Ramificación y poda para una matriz de',n,'elementos:')
    t1 = time.perf_counter()
    coste1 = ramificacion_y_poda(matriz)
    t1 = time.perf_counter() - t1
    
    # Medimos el tiempo de ejecución del algoritmo de fuerza bruta
    print('Algoritmo de Fuerza bruta para una matriz de',n,'elementos:')
    t2 = time.perf_counter()
    coste2 = coste_fuerza_bruta(matriz)
    t2 = time.perf_counter() - t2
    
    # Añadir los resultados al DataFrame
    df = pd.concat([df, pd.DataFrame({'n': [n], 'Algoritmo Ramificacion y Poda': [t1], 'Algoritmo Fuerza Bruta': [t2]})])

# Quitamos Index
df.index=[''] * len(df)

print()
print('TABLA DE TIEMPOS (en segundos')
display(df)

Algoritmo de Ramificación y poda para una matriz de 5 elementos:
La solucion final es: [{'s': (2, 3, 1, 0, 4), 'ci': 54}] en 21 iteraciones para dimension: 5
Algoritmo de Fuerza bruta para una matriz de 5 elementos:
La solucion final es: [{'s': (2, 3, 1, 0, 4), 'ci': 54}] en 120 iteraciones para dimension: 5
Algoritmo de Ramificación y poda para una matriz de 6 elementos:
La solucion final es: [{'s': (5, 4, 2, 0, 1, 3), 'ci': 80}] en 35 iteraciones para dimension: 6
Algoritmo de Fuerza bruta para una matriz de 6 elementos:
La solucion final es: [{'s': (5, 4, 2, 0, 1, 3), 'ci': 80}] en 720 iteraciones para dimension: 6
Algoritmo de Ramificación y poda para una matriz de 7 elementos:
La solucion final es: [{'s': (0, 5, 1, 4, 2, 6, 3), 'ci': 89}] en 59 iteraciones para dimension: 7
Algoritmo de Fuerza bruta para una matriz de 7 elementos:
La solucion final es: [{'s': (0, 5, 1, 4, 2, 6, 3), 'ci': 89}] en 5040 iteraciones para dimension: 7
Algoritmo de Ramificación y poda para una matriz de

,n,Algoritmo Ramificacion y Poda,Algoritmo Fuerza Bruta
,5,0.005145,0.003558
,6,0.001050,0.013534
,7,0.002851,0.091970
,8,0.009032,5.926579


Tras comparar el funcionamiento de ambos algoritmos, podemos sacar algunas conclusiones:

- El algoritmo de ramificación y poda siempre es más eficiente que el de fuerza bruta. El número de iteraciones es significativamente menor en todos los casos.
- Cuando la matriz es de 8x8, el tiempo de ejecución con los recursos de Google Colab ha sido de 0.009 s (113 iteraciones) para el de ramificación y poda, y de 5.927 s para el de fuerza bruta (40320 iteraciones).
- Para matrices más grandes de 8x8, el tiempo y recursos aumentan considerablemente. Estos valores pueden cambiar porque tratamos con matrices aleatorias, y dependiendo de éstas, hay cierta variación en los resultados.
- El orden de complejidad para el algoritmo de ramificación y poda suele ser de $O(n^2)$ y el de fuerza bruta de $O(n!)$ para este caso. No obstante, la poda permite descartar muchas soluciones parciales, haciendo el algoritmo más eficiente para este problema de asignación.